In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 22:18:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/23 22:18:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
import pandas as pd

In [4]:
from pyspark.sql import types

In [5]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

In [9]:
df_green_pd = pd.read_csv('data/raw/green/2021/01/green_tripdata_2021_01.csv.gz', nrows=1000)
df_green_pd

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1,43,151,1,1.01,5.5,0.5,0.5,0.00,0.00,NaN,0.3,6.80,2,1,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1,166,239,1,2.53,10.0,0.5,0.5,2.81,0.00,NaN,0.3,16.86,1,1,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1,41,42,1,1.12,6.0,0.5,0.5,1.00,0.00,NaN,0.3,8.30,1,1,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1,168,75,1,1.99,8.0,0.5,0.5,0.00,0.00,NaN,0.3,9.30,2,1,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2,265,265,3,0.00,-52.0,0.0,-0.5,0.00,0.00,NaN,-0.3,-52.80,3,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2021-01-02 12:48:43,2021-01-02 13:54:15,N,1,15,177,1,25.76,77.0,0.0,0.5,2.75,12.24,NaN,0.3,92.79,1,1,0.00
996,1,2021-01-02 12:16:47,2021-01-02 12:24:56,N,1,116,74,1,2.00,9.0,0.0,0.5,1.95,0.00,NaN,0.3,11.75,1,1,0.00
997,2,2021-01-02 12:07:17,2021-01-02 12:12:06,N,1,42,41,1,0.87,5.5,0.0,0.5,0.00,0.00,NaN,0.3,6.30,2,1,0.00
998,2,2021-01-02 12:32:45,2021-01-02 12:59:46,N,1,82,218,1,11.88,35.0,0.0,0.5,0.00,0.00,NaN,0.3,35.80,2,1,0.00


In [11]:
df_green_pd.dtypes

VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                 int64
PULocationID               int64
DOLocationID               int64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                  int64
congestion_surcharge     float64
dtype: object

In [6]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4
processing data for 2020/5
processing data for 2020/6
processing data for 2020/7


processing data for 2020/8
processing data for 2020/9


processing data for 2020/10


processing data for 2020/11
processing data for 2020/12


In [12]:
year = 2021 

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1
processing data for 2021/2
processing data for 2021/3
processing data for 2021/4
processing data for 2021/5
processing data for 2021/6


processing data for 2021/7
processing data for 2021/8
processing data for 2021/9


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/shawn/zoomcamp/week5/code/data/raw/green/2021/09.

In [13]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [14]:
year = 2021

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7


processing data for 2021/8
processing data for 2021/9


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/shawn/zoomcamp/week5/code/data/raw/yellow/2021/09.